## Follow tutorial for nlp-getting-started

## Import libraries

In [1]:
import io
import os
import sys
import time
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score

import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import text_classification, TextClassificationDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.utils import download_from_url, extract_archive, unicode_csv_reader
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.clean_text import clean_text
pd.set_option('display.max_colwidth', -1)

## Create torchdataset from csv

In [2]:
def _csv_iterator(data_path, ngrams, yield_cls=False):
    tokenizer = get_tokenizer("basic_english")
    with io.open(data_path, encoding="utf8") as f:
        reader = unicode_csv_reader(f)
        next(reader)
        for row in reader:
            tokens = ' '.join(row[1:4])
            tokens = clean_text(tokens)
            tokens = tokenizer(tokens)
            if yield_cls:
                yield int(row[4]), ngrams_iterator(tokens, ngrams)
            else:
                yield ngrams_iterator(tokens, ngrams)

def _create_data_from_iterator(vocab, iterator, include_unk):
    data = []
    labels = []
    with tqdm(unit_scale=0, unit='lines') as t:
        for cls, tokens in iterator:
            if include_unk:
                tokens = torch.tensor([vocab[token] for token in tokens])
            else:
                token_ids = list(filter(lambda x: x is not Vocab.UNK, [vocab[token]
                                        for token in tokens]))
                tokens = torch.tensor(token_ids)
            if len(tokens) == 0:
                logging.info('Row contains no tokens.')
            data.append((cls, tokens))
            labels.append(cls)
            t.update(1)
    return data, set(labels)

In [3]:
train_data_path = "../data/train.csv"
test_data_path = "../data/test.csv"
NGRAMS = 2

In [4]:
vocab = build_vocab_from_iterator(_csv_iterator(train_data_path, NGRAMS))
train_iterator = _csv_iterator(train_data_path, NGRAMS, yield_cls=True)
test_iterator = _csv_iterator(test_data_path, NGRAMS, yield_cls=False)
train_data_set, labels = _create_data_from_iterator(vocab, train_iterator, include_unk=False)

7613lines [00:00, 24804.58lines/s]
7613lines [00:00, 14892.23lines/s]


## Define and initiate the model

In [5]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [6]:
VOCAB_SIZE = len(vocab)
EMBED_DIM = 32
NUM_CLASS = len(labels)
BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(DEVICE)

## Functions used to generate batch

In [7]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

## Define functions to train the model and evaluate results

In [8]:
def train_func(sub_train_):
    # Train the model
    train_loss = 0
    y_true = np.array([])
    y_pred = np.array([])
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(DEVICE), offsets.to(DEVICE), cls.to(DEVICE)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        y_true = np.append(y_true, cls.numpy())
        y_pred = np.append(y_pred, output.argmax(1).numpy())
        
    # Adjust the learning rate
    scheduler.step()
    
    return train_loss / len(sub_train_), f1_score(y_true, y_pred)

def test(data_):
    loss = 0
    y_true = np.array([])
    y_pred = np.array([])
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(DEVICE), offsets.to(DEVICE), cls.to(DEVICE)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            y_true = np.append(y_true, cls.numpy())
            y_pred = np.append(y_pred, output.argmax(1).numpy())
    
    return loss / len(data_), f1_score(y_true, y_pred)

## Split the dataset and run the model

In [9]:
N_EPOCHS = 50
min_valid_loss = float('inf')

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_data_set) * 0.90)
sub_train_, sub_valid_ = random_split(train_data_set, [train_len, len(train_data_set) - train_len])

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_f1 = train_func(sub_train_)
    valid_loss, valid_f1 = test(sub_valid_)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    
    print('Epoch: %d' %(epoch + 1), "time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tF1: {train_f1 * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tF1: {valid_f1 * 100:.1f}%(valid)')

Epoch: 1 time in 0 minutes, 0 seconds
	Loss: 0.0420(train)	|	F1: 56.9%(train)
	Loss: 0.0011(valid)	|	F1: 72.2%(valid)
Epoch: 2 time in 0 minutes, 0 seconds
	Loss: 0.0257(train)	|	F1: 78.3%(train)
	Loss: 0.0042(valid)	|	F1: 66.2%(valid)
Epoch: 3 time in 0 minutes, 0 seconds
	Loss: 0.0164(train)	|	F1: 87.9%(train)
	Loss: 0.0011(valid)	|	F1: 74.6%(valid)
Epoch: 4 time in 0 minutes, 0 seconds
	Loss: 0.0100(train)	|	F1: 94.3%(train)
	Loss: 0.0002(valid)	|	F1: 64.7%(valid)
Epoch: 5 time in 0 minutes, 0 seconds
	Loss: 0.0065(train)	|	F1: 96.7%(train)
	Loss: 0.0016(valid)	|	F1: 74.9%(valid)
Epoch: 6 time in 0 minutes, 0 seconds
	Loss: 0.0049(train)	|	F1: 97.6%(train)
	Loss: 0.0015(valid)	|	F1: 75.9%(valid)
Epoch: 7 time in 0 minutes, 0 seconds
	Loss: 0.0041(train)	|	F1: 98.0%(train)
	Loss: 0.0008(valid)	|	F1: 75.2%(valid)
Epoch: 8 time in 0 minutes, 0 seconds
	Loss: 0.0038(train)	|	F1: 98.4%(train)
	Loss: 0.0010(valid)	|	F1: 75.0%(valid)
Epoch: 9 time in 0 minutes, 0 seconds
	Loss: 0.0035(trai

## Train model on entire dataset
 - expected F1 score ~ 76.1%

In [11]:
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(DEVICE)
criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_f1 = train_func(train_data_set)
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60
    
    print('Epoch: %d' %(epoch + 1), "time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tF1: {train_f1 * 100:.1f}%(train)')

Epoch: 1 time in 0 minutes, 0 seconds
	Loss: 0.0417(train)	|	F1: 57.6%(train)
Epoch: 2 time in 0 minutes, 0 seconds
	Loss: 0.0254(train)	|	F1: 78.5%(train)
Epoch: 3 time in 0 minutes, 0 seconds
	Loss: 0.0162(train)	|	F1: 88.0%(train)
Epoch: 4 time in 0 minutes, 0 seconds
	Loss: 0.0095(train)	|	F1: 94.2%(train)
Epoch: 5 time in 0 minutes, 0 seconds
	Loss: 0.0066(train)	|	F1: 96.7%(train)
Epoch: 6 time in 0 minutes, 0 seconds
	Loss: 0.0056(train)	|	F1: 97.4%(train)
Epoch: 7 time in 0 minutes, 0 seconds
	Loss: 0.0042(train)	|	F1: 98.1%(train)
Epoch: 8 time in 0 minutes, 0 seconds
	Loss: 0.0040(train)	|	F1: 98.2%(train)
Epoch: 9 time in 0 minutes, 0 seconds
	Loss: 0.0040(train)	|	F1: 98.3%(train)
Epoch: 10 time in 0 minutes, 0 seconds
	Loss: 0.0033(train)	|	F1: 98.4%(train)
Epoch: 11 time in 0 minutes, 0 seconds
	Loss: 0.0031(train)	|	F1: 98.5%(train)
Epoch: 12 time in 0 minutes, 0 seconds
	Loss: 0.0028(train)	|	F1: 98.6%(train)
Epoch: 13 time in 0 minutes, 0 seconds
	Loss: 0.0026(train)	|

## Evaluate the model with test dataset

In [12]:
def _create_test_data_from_iterator(vocab, iterator, include_unk):
    data = []
    with tqdm(unit_scale=0, unit='lines') as t:
        for tokens in iterator:
            if include_unk:
                tokens = torch.tensor([vocab[token] for token in tokens])
            else:
                token_ids = list(filter(lambda x: x is not Vocab.UNK, [vocab[token]
                                        for token in tokens]))
                tokens = torch.tensor(token_ids)
            if len(tokens) == 0:
                logging.info('Row contains no tokens.')
            data.append(tokens)
            t.update(1)
    return data

In [13]:
test_data_set = _create_test_data_from_iterator(vocab, test_iterator, include_unk=False)

3263lines [00:00, 15151.72lines/s]


In [18]:
def predict(text, model):
    with torch.no_grad():
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

In [19]:
test_y_pred = [predict(i, model) for i in test_data_set]

In [32]:
test = pd.read_csv("../data/test.csv")
test["target"] = test_y_pred
test_submit = test[["id", "target"]]
test_submit.to_csv("../builds/submission/baseline.csv", index=False)

In [33]:
test.shape

(3263, 5)